In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models


In [2]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])




In [3]:
trainset = datasets.CIFAR10(root='./data', train=True, download=False, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=4)
testset = datasets.CIFAR10(root='./data', train=False, download=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True, num_workers=4)


In [4]:
class SimpleResNet(nn.Module):
    def __init__(self, num_classes=10, dropout_prob=0.5):
        super(SimpleResNet, self).__init__()
        
        # Initial Layer
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(64)

        # Res Block 1
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)

        # Res Block 2 with downsample
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.res3 = nn.Conv2d(64, 128, kernel_size=1, stride=2)

        # Res Block 3
        self.conv4 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(128)

        # Res Block 4 with downsample
        self.conv5 = nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1)
        self.bn5 = nn.BatchNorm2d(256)
        self.res5 = nn.Conv2d(128, 256, kernel_size=1, stride=2)

        # Dropout layer
        self.dropout = nn.Dropout(dropout_prob)

        # FC layers
        self.pool = nn.MaxPool2d(2,2)
        self.fc1 = nn.Linear(256 * 4 * 4, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))

        # Block 1
        identity = x
        out = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(out + identity)

        # Block 2
        identity = self.res3(x)
        out = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(out + identity)

        # Block 3
        identity = x
        out = F.relu(self.bn4(self.conv4(x)))
        x = F.relu(out + identity)

        # Block 4
        identity = self.res5(x)
        out = F.relu(self.bn5(self.conv5(x)))
        x = F.relu(out + identity)

        # Max Pool
        x = self.pool(x)

        # Flatten and apply dropout
        x = x.view(x.size(0), -1)
        x = self.dropout(x)

        # Additional FC layer with dropout
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)

        return x

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleResNet().to(device)
print(next(model.parameters()).device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.0001)


cuda:0


In [ ]:
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, (images, labels) in enumerate(trainloader):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(trainloader):.4f}")

    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f"Test Accuracy: {100 * correct / total:.2f}%")


 


Epoch [1/20], Loss: 1.6005
Test Accuracy: 55.13%
Epoch [2/20], Loss: 1.2550
Test Accuracy: 62.27%
Epoch [3/20], Loss: 1.1064
Test Accuracy: 68.49%
Epoch [4/20], Loss: 1.0116
Test Accuracy: 71.66%
Epoch [5/20], Loss: 0.9363
Test Accuracy: 73.52%
Epoch [6/20], Loss: 0.8791
Test Accuracy: 75.66%
Epoch [7/20], Loss: 0.8320
Test Accuracy: 76.97%
Epoch [8/20], Loss: 0.7878
Test Accuracy: 77.90%
Epoch [9/20], Loss: 0.7493
Test Accuracy: 78.01%
Epoch [10/20], Loss: 0.7193
Test Accuracy: 78.00%
Epoch [11/20], Loss: 0.6874
Test Accuracy: 80.17%
